# <font color="yellow">Remarks</font>
This notebook is based on a Tutorial notebook on the tensorflow web site.

1.	Custom training: basics  |  TensorFlow. TensorFlow. https://www.tensorflow.org/tutorials/eager/custom_training. Published December 17, 2018. Accessed December 29, 2018.

You can find some more insformation on eager execution, also on Tensorflow's website:
2.	Eager Execution  |  TensorFlow. TensorFlow. https://www.tensorflow.org/guide/eager. Published December 12, 2018. Accessed December 29, 2018.



<font color="yellow">We have marked sections that have differences with the original workbook in yellow</font>.
    
Author: H. Felix Wittmann
hfwittmann@gmail.com

There is no association between the author and Tensorflow.


##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Custom training: basics

In the previous tutorial we covered the TensorFlow APIs for automatic differentiation, a basic building block for machine learning.
In this tutorial we will use the TensorFlow primitives introduced in the prior tutorials to do some simple machine learning.

TensorFlow also includes a higher-level neural networks API (`tf.keras`) which provides useful abstractions to reduce boilerplate. We strongly recommend those higher level APIs for people working with neural networks. However, in this short tutorial we cover neural network training from first principles to establish a strong foundation.

## Setup

In [0]:
import tensorflow as tf

tf.enable_eager_execution()

## Variables

Tensors in TensorFlow are immutable stateless objects. Machine learning models, however, need to have changing state: as your model trains, the same code to compute predictions should behave differently over time (hopefully with a lower loss!). To represent this state which needs to change over the course of your computation, you can choose to rely on the fact that Python is a stateful programming language:


In [0]:
# Using python state
x = tf.zeros([10, 10])
x += 2  # This is equivalent to x = x + 2, which does not mutate the original
        # value of x
print(x)

TensorFlow, however, has stateful operations built in, and these are often more pleasant to use than low-level Python representations of your state. To represent weights in a model, for example, it's often convenient and efficient to use TensorFlow variables.

A Variable is an object which stores a value and, when used in a TensorFlow computation, will implicitly read from this stored value. There are operations (`tf.assign_sub`, `tf.scatter_update`, etc) which manipulate the value stored in a TensorFlow variable.

In [0]:
v = tf.Variable(1.0)
assert v.numpy() == 1.0

# Re-assign the value
v.assign(3.0)
assert v.numpy() == 3.0

# Use `v` in a TensorFlow operation like tf.square() and reassign
v.assign(tf.square(v))
assert v.numpy() == 9.0

Computations using Variables are automatically traced when computing gradients. For Variables representing embeddings TensorFlow will do sparse updates by default, which are more computation and memory efficient.

Using Variables is also a way to quickly let a reader of your code know that this piece of state is mutable.

## Example: Fitting a linear model

Let's now put the few concepts we have so far ---`Tensor`, `GradientTape`, `Variable` --- to build and train a simple model. This typically involves a few steps:

1. Define the model.
2. Define a loss function.
3. Obtain training data.
4. Run through the training data and use an "optimizer" to adjust the variables to fit the data.

In this tutorial, we'll walk through a trivial example of a simple linear model: `f(x) = x * W + b`, which has two variables - `W` and `b`. Furthermore, we'll synthesize data such that a well trained model would have `W = 3.0` and `b = 2.0`.

### Define the model

Let's define a simple class to encapsulate the variables and the computation.

In [0]:
class Model(object):
  def __init__(self):
    # Initialize variable to (5.0, 0.0)
    # In practice, these should be initialized to random values.
    self.W = tf.Variable(5.0)
    self.b = tf.Variable(0.0)
    
  def __call__(self, x):
    return self.W * x + self.b
  
model = Model()

assert model(3.0).numpy() == 15.0

### Define a loss function

A loss function measures how well the output of a model for a given input matches the desired output. Let's use the standard L2 loss.

In [0]:
def loss(predicted_y, desired_y):
  return tf.reduce_mean(tf.square(predicted_y - desired_y))

### <font color='yellow'>Obtain training data</font>

Let's synthesize the training data with some noise.

In [0]:
import numpy as np
nOfDatapoints = 1000
W_TRUE, b_TRUE = 3.0, 2.0
np.random.seed(314)
inputs = np.random.normal(size=nOfDatapoints)
noise = np.random.normal(size=nOfDatapoints)
outputs = W_TRUE * inputs + b_TRUE + noise

### <font color='yellow'>Split into train/test</font>

In [0]:
from sklearn.model_selection import train_test_split
inputs_train, inputs_test, outputs_train, outputs_test = \
    train_test_split(inputs, outputs, test_size=0.33, random_state=42)
# outputs = outputs.reshape(-1,1)  # reshape to match keras output

In [0]:
import matplotlib.pyplot as plt
plt.scatter(inputs,outputs,c='b', alpha=0.5, label='Data')
plt.scatter(inputs_train, outputs_train, c='y', alpha=0.2, label='Train')
plt.scatter(inputs_test, outputs_test, c='g', alpha=0.2, label='Test')
plt.legend()## Explore data und Model

model = Model()

print('Current loss: '),
print(loss(model(inputs), outputs).numpy())

### <font color='yellow'>Explore data und Model before training</font>

Before we train the model let's visualize where the model stands right now. We'll plot the model's predictions in black and the testting data in blue.

In [0]:
model = Model()

prediction_test = model(inputs_test).numpy()
plt.scatter(inputs_test, outputs_test, c = 'b', alpha=0.5, label = 'Test Data')
plt.scatter(inputs_test, prediction_test, c = 'k', alpha=0.5, label = 'Initial Model Guess')
plt.legend()
plt.show()

### Define a training loop

We now have our network and our training data. Let's train it, i.e., use the training data to update the model's variables (`W` and `b`) so that the loss goes down using [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent). There are many variants of the gradient descent scheme that are captured in `tf.train.Optimizer` implementations. We'd highly recommend using those implementations, but in the spirit of building from first principles, in this particular example we will implement the basic math ourselves.

### <font color='yellow'> Define gradient tape </font>

In [0]:
def grad(model, inputs, outputs):
    with tf.GradientTape() as tape:
        current_loss = loss(model(inputs), outputs)
    return current_loss, tape.gradient(current_loss, [model.W, model.b] )

Finally, let's repeatedly run through the training data and see how `W` and `b` evolve.

In [0]:
learning_rate = 0.1
model = Model()

# Collect the history of W-values and b-values to plot later
Ws, bs, losses = [], [], []
epochs = range(10)
for epoch in epochs:
    Ws.append(model.W.numpy())
    bs.append(model.b.numpy())

    
    current_loss, [dW, db] = grad(model, inputs_train, outputs_train)
    
    losses.append(current_loss)
    
    model.W.assign_add(-learning_rate*dW)
    model.b.assign_add(-learning_rate*db)


    print('Epoch %2d: W=%1.2f b=%1.2f, loss=%2.5f' %
        (epoch, Ws[-1], bs[-1], current_loss))

# Let's plot it all
plt.plot(epochs, Ws, 'r',
         epochs, bs, 'b')
plt.plot([W_TRUE] * len(epochs), 'r--',
         [b_TRUE] * len(epochs), 'b--')
plt.legend(['W', 'b', 'true W', 'true_b'])
plt.show()
  

### <font color='yellow'>A more abstract representation is the loss function</font>

In [0]:
# Let's plot the history
plt.plot(losses)
plt.title('Evolution of loss')

In [0]:
# Let's plot the history
plt.plot(losses)
plt.title('Evolution of loss')

## Next Steps

In this tutorial we covered `Variable`s and built and trained a simple linear model using the TensorFlow primitives discussed so far.

In theory, this is pretty much all you need to use TensorFlow for your machine learning research.
In practice, particularly for neural networks, the higher level APIs like `tf.keras` will be much more convenient since it provides higher level building blocks (called "layers"), utilities to save and restore state, a suite of loss functions, a suite of optimization strategies etc.


## <font color="yellow">Explore data und Model after training</font>

In [0]:
# model = Model()

prediction_test = model(inputs_test).numpy()
plt.scatter(inputs_test, outputs_test, c = 'b', alpha=0.5, label = 'Test Data')
plt.scatter(inputs_test, prediction_test, c = 'k', alpha=0.5, label = 'Model Prediction')
plt.legend()
plt.show()

## <font color='yellow'>Explore loss function</font>

In [0]:
def myloss(W,b):
    return ((inputs_train * W + b - outputs_train) ** 2).mean()

##  <font color='yellow'>Calculate loss surface</font>

In [0]:
mean = [3,2]
cov = [[2,0],[0,2]]
Wb = np.random.multivariate_normal(mean=mean, cov=cov, size=10000)
W, b = Wb[:,0], Wb[:,1]
loss = [myloss(W,b) for (W,b) in Wb]

## Prepare plotting

In [0]:
# https://colab.research.google.com/notebooks/charts.ipynb#scrollTo=Xbey0KVpfIbQ
from plotly.offline import iplot, plot, init_notebook_mode
import plotly.graph_objs as go


def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
  '''))
  init_notebook_mode(connected=False)

get_ipython().events.register('pre_run_cell', enable_plotly_in_cell)

### <font color='yellow'>Calculate the trace for the surface data</font>

In [0]:
trace = go.Scatter3d(
    x = W,
    y = b,
    z = loss,
    mode='markers',
    marker = dict(
        opacity = 0.5,
        size = 2,
        color = np.sin(loss)
    ),
    name = 'General Surface'
)

data = [trace]
layout = go.Layout(
    title='Loss function',
    scene = dict (
        xaxis =  dict(title='W'),
        yaxis = dict(title='b'),
        zaxis = dict(title='loss')
    )
)


### <font color='yellow'> Calculate the trace for the steps of the gradient based optimisation</font>

In [0]:
trace2 = go.Scatter3d(
    x = Ws,
    y = bs,
    z = np.array(losses),
    mode='lines+markers',
    marker = dict(
        opacity = 0.5,
        size = 5,
        color = 'black'
    ),
    name = 'Gradient-based Optimisation'
)

data = [trace, trace2]

layout = go.Layout(
    title='Loss function',
    scene = dict (
        xaxis =  dict(title='W'),
        yaxis = dict(title='b'),
        zaxis = dict(title='loss')
    )
)

iplot({'data':data, 'layout': layout})
# cplot({'data':data, 'layout': layout},'loss function.html')